In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
# Import Libraries
import pandas as pd
from pyspark.ml.feature import StringIndexer
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import FloatType, LongType
from sklearn.model_selection import train_test_split

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('future.no_silent_downcasting', True)

print("Libraries imported successfully")
print(f"Pandas version: {pd.__version__}")


In [ ]:
# Initialize Spark
spark = SparkSession.builder \
    .appName("AirbnbAnalysis") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .getOrCreate()

print("Spark session created")
print(f"Spark version: {spark.version}")
spark

In [ ]:
# Configuration Parameters
class Config:
    # File paths
    LISTINGS_PATH = '../data/listings.csv'
    REVIEWS_PATH = '../data/reviews.csv'
    OUTPUT_DIR = '../data/'

    # Filtering thresholds
    MIN_USER_REVIEWS = 3  # Minimum reviews per user
    MIN_LISTING_REVIEWS = 5  # Minimum reviews per listing

    # Train/test split
    TRAIN_RATIO = 0.8
    RANDOM_SEED = 42


config = Config()
print("Configuration set")

In [ ]:
# Load Raw Data
print("Loading data...")

listings_cols = [
    'id', 'name', 'description', 'neighbourhood_cleansed',
    'latitude', 'longitude', 'property_type', 'room_type',
    'accommodates', 'bedrooms', 'beds', 'price',
    'minimum_nights', 'number_of_reviews', 'review_scores_rating',
    'review_scores_location', 'review_scores_value',
    'host_is_superhost', 'instant_bookable', 'picture_url'
]

reviews_cols = ['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments']

listings_raw = pd.read_csv(config.LISTINGS_PATH, usecols=listings_cols)
reviews_raw = pd.read_csv(config.REVIEWS_PATH, usecols=reviews_cols)

print(f"✓ Loaded {len(listings_raw):,} listings")
print(f"✓ Loaded {len(reviews_raw):,} reviews")
print(f"✓ Unique users: {reviews_raw['reviewer_id'].nunique():,}")
print(f"✓ Unique listings: {reviews_raw['listing_id'].nunique():,}")

In [ ]:
# Visualize Distributions
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Reviews per user
user_review_counts = reviews_raw['reviewer_id'].value_counts()
axes[0, 0].hist(user_review_counts, bins=range(1, 7), edgecolor='black')
axes[0, 0].set_xlabel('Number of Reviews')
axes[0, 0].set_ylabel('Number of Users')
axes[0, 0].set_title('Reviews per User Distribution')
axes[0, 0].axvline(config.MIN_USER_REVIEWS, color='red', linestyle='--',
                   label=f'Threshold: {config.MIN_USER_REVIEWS}')
axes[0, 0].set_xlim(1, 6)
axes[0, 0].legend()

# Reviews per listing
listing_review_counts = reviews_raw['listing_id'].value_counts()
axes[0, 1].hist(listing_review_counts, bins=range(1, 26), edgecolor='black')
axes[0, 1].set_xlabel('Number of Reviews')
axes[0, 1].set_ylabel('Number of Listings')
axes[0, 1].set_title('Reviews per Listing Distribution')
axes[0, 1].axvline(config.MIN_LISTING_REVIEWS, color='red', linestyle='--',
                   label=f'Threshold: {config.MIN_LISTING_REVIEWS}')
axes[0, 1].set_xlim(1, 25)
axes[0, 1].legend()

# Reviews over time
reviews_raw['date'] = pd.to_datetime(reviews_raw['date'])
reviews_by_month = reviews_raw.set_index('date').resample('ME').size()
axes[1, 0].plot(reviews_by_month[:-1])
axes[1, 0].set_xlabel('Date')
axes[1, 0].set_ylabel('Number of Reviews')
axes[1, 0].set_title('Reviews Over Time')
axes[1, 0].tick_params(axis='x', rotation=45)

# Price distribution
axes[1, 1].hist(listings_raw['price'].str.replace('$', '').str.replace(',', '').astype(float).dropna().clip(0, 670),
                bins=50, edgecolor='black')
axes[1, 1].set_xlabel('Price ($)')
axes[1, 1].set_ylabel('Number of Listings')
axes[1, 1].set_title('Price Distribution (capped at $670)')

plt.tight_layout()
plt.show()

print(f"Users with ≥{config.MIN_USER_REVIEWS} reviews: {(user_review_counts >= config.MIN_USER_REVIEWS).sum():,}")
print(
    f"Listings with ≥{config.MIN_LISTING_REVIEWS} reviews: {(listing_review_counts >= config.MIN_LISTING_REVIEWS).sum():,}")


In [ ]:
# Clean Listings Data
listings_clean = listings_raw.copy()

# 1. Clean price
listings_clean['price'] = listings_clean['price'].str.replace('$', '').str.replace(',', '').astype(float)

# 2. Handle missing values

# listings_clean['review_scores_rating'].fillna(
#     listings_clean['review_scores_rating'].median(),
#     inplace=True
# )

listings_clean['bedrooms'] = listings_clean['bedrooms'].fillna(0)
listings_clean['beds'] = listings_clean['beds'].fillna(0)

# 3. Convert boolean columns
listings_clean['host_is_superhost'] = listings_clean['host_is_superhost'].map({'t': True, 'f': False}).fillna(False)
listings_clean['instant_bookable'] = listings_clean['instant_bookable'].map({'t': True, 'f': False}).fillna(False)

# 4. Rename id column for clarity
listings_clean.rename(columns={'id': 'listing_id'}, inplace=True)

print("✓ Listings data cleaned")
print(f"Price range: ${listings_clean['price'].min():.2f} - ${listings_clean['price'].max():.2f}")
print(f"Median price: ${listings_clean['price'].median():.2f}")


In [ ]:
# Cell 9: Apply Filtering
print("Applying filters...")
print(f"Original: {len(reviews_raw):,} reviews")

# Count reviews per user and listing
user_counts = reviews_raw['reviewer_id'].value_counts()
listing_counts = reviews_raw['listing_id'].value_counts()

# Identify active users and popular listings
active_users = user_counts[user_counts >= config.MIN_USER_REVIEWS].index
popular_listings = listing_counts[listing_counts >= config.MIN_LISTING_REVIEWS].index

print(f"Active users (≥{config.MIN_USER_REVIEWS} reviews): {len(active_users):,}")
print(f"Popular listings (≥{config.MIN_LISTING_REVIEWS} reviews): {len(popular_listings):,}")

# Apply filters
reviews_filtered = reviews_raw[
    reviews_raw['reviewer_id'].isin(active_users) &
    reviews_raw['listing_id'].isin(popular_listings)
    ].copy()

reviews_filtered = reviews_filtered.sort_values('date').drop_duplicates(
    subset=['reviewer_id', 'listing_id'],
    keep='last'
)

print(f"Filtered: {len(reviews_filtered):,} reviews")
print(f"Reduction: {(1 - len(reviews_filtered) / len(reviews_raw)) * 100:.1f}%")

# Update listings to only include those with reviews
listings_filtered = listings_clean[
    listings_clean['listing_id'].isin(reviews_filtered['listing_id'])
].copy()

print(f"Listings after filtering: {len(listings_filtered):,}")

# Calculate sparsity
n_users = reviews_filtered['reviewer_id'].nunique()
n_listings = reviews_filtered['listing_id'].nunique()
n_interactions = len(reviews_filtered)
sparsity = 1 - (n_interactions / (n_users * n_listings))

print(f"\nDataset Statistics:")
print(f"  Users: {n_users:,}")
print(f"  Listings: {n_listings:,}")
print(f"  Interactions: {n_interactions:,}")
print(f"  Sparsity: {sparsity * 100:.2f}%")


In [ ]:
# Construct Ratings

def construct_continuous_ratings(reviews_df):
    """
    Airbnb implicit rating construction from review engagement signals
    """
    df = reviews_df.copy()
    df['date'] = pd.to_datetime(df['date'])
    df['review_length'] = df['comments'].fillna('').astype(str).str.len()

    max_date = df['date'].max()
    df['days_ago'] = (max_date - df['date']).dt.days

    # User engagement: How active is this reviewer?
    user_activity = df.groupby('reviewer_id').size().reset_index(name='user_review_count')
    df = df.merge(user_activity, on='reviewer_id', how='left')

    # Listing popularity: How many reviews does this property have?
    listing_popularity = df.groupby('listing_id').size().reset_index(name='listing_review_count')
    df = df.merge(listing_popularity, on='listing_id', how='left')

    # Feature engineering
    df['recency_score'] = np.exp(-df['days_ago'] / 365)  # 1-year decay (faster for travel)
    df['length_score'] = np.log1p(df['review_length'])
    df['user_credibility'] = np.log1p(df['user_review_count'])  # Experienced reviewers more valuable
    df['listing_signal'] = np.log1p(df['listing_review_count'])  # Popular listings get boost

    # Normalize to 0-1
    df['recency_norm'] = df['recency_score']  # Already 0-1
    df['length_norm'] = df['length_score'] / df['length_score'].max()
    df['credibility_norm'] = df['user_credibility'] / df['user_credibility'].max()
    df['popularity_norm'] = df['listing_signal'] / df['listing_signal'].max()

    # Airbnb-optimized weights
    df['composite_score'] = (
            df['recency_norm'] * 0.5 +  # Highest: Properties change quickly
            df['length_norm'] * 0.3 +  # Strong signal: Detailed = satisfied
            df['credibility_norm'] * 0.1 +  # Moderate: Trust experienced reviewers
            df['popularity_norm'] * 0.1  # Moderate: Popular listings have reasons
    )

    # Map to 1-5 scale
    df['rating'] = (df['composite_score'].rank(method='average', pct=True) * 4 + 1).round(2)

    return df


reviews_with_ratings = construct_continuous_ratings(reviews_filtered)

print(f"\nFiltered Rating distribution:")
print(reviews_with_ratings['rating'].describe())

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
axes[0].hist(reviews_with_ratings['rating'], bins=30, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Rating')
axes[0].set_ylabel('Frequency')
axes[0].axvline(reviews_with_ratings['rating'].mean(), color='red', linestyle='--',
                label=f'Mean: {reviews_with_ratings["rating"].mean():.2f}')
axes[0].legend()

# Value counts
rating_counts = reviews_with_ratings['rating'].value_counts().sort_index()
axes[1].bar(rating_counts.index, rating_counts.values, edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Rating')
axes[1].set_ylabel('Count')
axes[1].set_title('Rating Value Counts')

plt.tight_layout()
plt.show()

In [ ]:
# Create Final Interactions DataFrame
interactions = reviews_with_ratings[[
    'reviewer_id',
    'listing_id',
    'rating',
    'date'
]].copy()

# Remove duplicates (keep most recent review if user reviewed same listing multiple times)
# interactions = interactions.sort_values('date').drop_duplicates(
#     subset=['reviewer_id', 'listing_id'],
#     keep='last'
# )

print(f"✓ Interactions DataFrame created")
print(f"Shape: {interactions.shape}")
print(f"\nSample:")
print(interactions.head())

In [ ]:
interactions

In [ ]:
# Train/Test Split

# Split by user (ensure each user appears in both train and test)
train_data, test_data = train_test_split(
    interactions,
    test_size=(1 - config.TRAIN_RATIO),
    random_state=config.RANDOM_SEED,
    stratify=None  # Random split
)

print(f"✓ Data split into train/test")
print(f"Training set: {len(train_data):,} interactions")
print(f"Test set: {len(test_data):,} interactions")
print(
    f"Split ratio: {len(train_data) / len(interactions) * 100:.1f}% / {len(test_data) / len(interactions) * 100:.1f}%")

# Verify users/listings overlap
train_users = set(train_data['reviewer_id'])
test_users = set(test_data['reviewer_id'])
train_listings = set(train_data['listing_id'])
test_listings = set(test_data['listing_id'])

print(f"\nUsers in both sets: {len(train_users & test_users):,}")
print(f"Listings in both sets: {len(train_listings & test_listings):,}")


In [ ]:
# Convert to Spark DataFrames

# Convert interactions to Spark
train_spark = spark.createDataFrame(train_data)
test_spark = spark.createDataFrame(test_data)

# Convert listings to Spark
listings_spark = spark.createDataFrame(listings_filtered)

print("✓ Converted to Spark DataFrames")
print("\nTrain schema:")
train_spark.printSchema()
print(f"\nTrain count: {train_spark.count():,}")

In [ ]:
# Create Integer IDs for ALS

# 1. Combine train and test for indexer fitting
combined_ids = train_spark.select("reviewer_id", "listing_id").union(
    test_spark.select("reviewer_id", "listing_id")
).distinct()

# 2. Fit indexers on ALL unique IDs
user_indexer = StringIndexer(inputCol="reviewer_id", outputCol="user_id", )
item_indexer = StringIndexer(inputCol="listing_id", outputCol="item_id", )

user_indexer_model = user_indexer.fit(combined_ids.select("reviewer_id"))
item_indexer_model = item_indexer.fit(combined_ids.select("listing_id"))

# 3. Transform train and test
train_indexed = user_indexer_model.transform(train_spark)
train_indexed = item_indexer_model.transform(train_indexed)

test_indexed = user_indexer_model.transform(test_spark)
test_indexed = item_indexer_model.transform(test_indexed)

# Cast to appropriate types
train_final = train_indexed.select(
    col("user_id").cast(LongType()),
    col("item_id").cast(LongType()),
    col("rating").cast(FloatType()),
    col("reviewer_id"),  # Keep original IDs for reference
    col("listing_id")
)

test_final = test_indexed.select(
    col("user_id").cast(LongType()),
    col("item_id").cast(LongType()),
    col("rating").cast(FloatType()),
    col("reviewer_id"),
    col("listing_id")
)

print("✓ User and item IDs indexed")
print("\nFinal train schema:")
train_final.printSchema()
print("\nSample:")
train_final.show(5)

In [ ]:
# Save ID Mappings for Later Use
# Extract user ID mapping
user_mapping = train_indexed.select("reviewer_id", "user_id").distinct().toPandas()
user_mapping.to_csv(f'{config.OUTPUT_DIR}user_id_mapping.csv', index=False)

# Extract item ID mapping
item_mapping = train_indexed.select("listing_id", "item_id").distinct().toPandas()
item_mapping.to_csv(f'{config.OUTPUT_DIR}item_id_mapping.csv', index=False)

print(f"✓ Saved user mapping: {len(user_mapping):,} users")
print(f"✓ Saved item mapping: {len(item_mapping):,} items")

In [ ]:
# Cell 15: Save All Processed Data

os.makedirs(config.OUTPUT_DIR, exist_ok=True)

# Save Spark DataFrames as Parquet
train_final.write.mode('overwrite').parquet(f'{config.OUTPUT_DIR}train.parquet')
test_final.write.mode('overwrite').parquet(f'{config.OUTPUT_DIR}test.parquet')
listings_spark.write.mode('overwrite').parquet(f'{config.OUTPUT_DIR}listings.parquet')

# Also save as CSV for easy inspection
train_final.limit(1000).toPandas().to_csv(f'{config.OUTPUT_DIR}train_sample.csv', index=False)
listings_filtered.to_csv(f'{config.OUTPUT_DIR}listings_metadata.csv', index=False)

print("✓ All data saved successfully")
print(f"\nFiles saved to: {config.OUTPUT_DIR}")
print("  - train.parquet")
print("  - test.parquet")
print("  - listings.parquet")
print("  - user_id_mapping.csv")
print("  - item_id_mapping.csv")
print("  - train_sample.csv")
print("  - listings_metadata.csv")